In [2]:
# MIT image
# Made by : HERO Kwon
# Date : 190108

import os
import numpy as np
import pandas as pd
import pickle
import gzip
import matplotlib.pyplot as plt
import math
import cmath

# data path
#path_meta = '/home/herokwon/mount_data/Data/Wi-Fi_meta/'
#path_csi = '/home/herokwon/mount_data/Data/Wi-Fi_processed/'
#path_csi_np = '/home/herokwon/mount_data/Data/Wi-Fi_processed_npy/'
#path_mit_image = '/home/herokwon/mount_data/Data/Wi-Fi_mit_denoise/'

# data path_mi
path_csi = 'D:\\Data\\Wi-Fi_processed\\'
path_csi_np = 'D:\\Data\\Wi-Fi_processed_npy\\'
path_meta = 'D:\\Data\\Wi-Fi_meta\\'
path_sc = 'D:\\Data\\Wi-Fi_info\\'
path_mit_image = 'D:\\Data\\Wi-Fi_mit_image\\'

# data info
df_info = pd.read_csv(path_meta+'data_subc_sig_v1.csv') # 55429 rows
#df_info = df_info[df_info.id_location==1]

#df_info = df_info.iloc[0:3700]
#df_info = df_info.iloc[3700:7400]
#df_info = df_info.iloc[7400:11100]
#df_info = df_info.iloc[11100:14800]
#df_info = df_info.iloc[14800:18500]
#df_info = df_info.iloc[18500:22200]
#df_info = df_info.iloc[22200:25900]
#df_info = df_info.iloc[25900:29600]
#df_info = df_info.iloc[29600:33300]
#df_info = df_info.iloc[33300:37000]
#df_info = df_info.iloc[37000:40700]
#df_info = df_info.iloc[40700:44400]
#df_info = df_info.iloc[44400:48100]
#df_info = df_info.iloc[48100:51800]
df_info = df_info.iloc[51800:55429]


person_uid = np.unique(df_info['id_person'])
dict_id = dict(zip(person_uid,np.arange(len(person_uid))))
csi_time = 100 #15000 #int(np.max(df_info['len']))
# parameters
max_value = np.max(df_info['max'].values)
#no_classes = len(np.unique(df_info['id_person']))
no_classes = len(dict_id)
csi_subc = 30
input_shape = (csi_time, csi_subc, 6)

# freq BW list
bw_list = pd.read_csv(path_meta+'wifi_f_bw_list.csv')
'''
# avg Array
with open(path_meta + 'dict_avgcsi.pickle','rb') as f:
    dict_avg = pickle.load(f)
'''
# 3D scan param
m,n = 2,3
c =  299792458 # speed of light 
#r = (160 + 160 + 164) * 0.01 # meter
r = 1.64 #meter
d = 45 * 0.01 # meter
ch = 6#3
max_subc = 30


# make data generator
def gen_csi(df_info,id_num,len_num):
    infiles = set([file.replace(".npy","") for file in os.listdir(path_mit_image)])
    tofiles = list(set(np.unique(df_info.id.values)) - infiles)
    for file in tofiles:    
        # Label
        id_key = df_info[df_info.id==file][['id_person','id_location','id_direction','id_exp']].values[0].astype('int')
        data1_y = dict_id[id_key[0]]

        # read sample data
        # load and uncompress.
        with gzip.open(path_csi+file+'.pickle.gz','rb') as f:
            data1 = pickle.load(f)

        s_idx = (np.arange(len_num) * data1.shape[0] / len_num).astype('int')
        dt=np.min([100,len(data1)-s_idx[-1]])
        list_avgt = []
        for idx in s_idx:
            data1_t = data1[idx:idx+dt,:,:,:]
            data1_avgt = np.mean(data1_t,axis=0)
            list_avgt.append(data1_avgt)
        data1_s = np.array(list_avgt)
        # zero pad
        #pad_len = len_num - data1_diff.shape[0]
        #data1_pad = np.pad(data1_diff,((0,pad_len),(0,0),(0,0),(0,0)),'constant',constant_values=0)

        # subcarrier info
        #data1_sc_df = pd.read_csv(path_sc + file + '_df_sc.csv')
        #data1_time = np.diff(data1_sc_df['timestamp_low'])
        #data1_time_pad = np.pad(data1_time,((1,pad_len)),'constant',constant_values=0)

        yield(data1_s ,data1_y,id_key,file)#,data1_time_pad)

gen = gen_csi(df_info,no_classes,csi_time)


In [3]:
target_sig,target_lab,target_id,target_file = next(gen)

In [5]:
target_sig.shape

(100, 30, 2, 3)

In [34]:
np.angle(target_sig)

array([[[[ 1.3334861 ,  0.46428844, -0.42361587],
         [-1.25635701, -2.08720476, -2.8919463 ]],

        [[ 2.64123406,  1.90715161,  1.15175303],
         [ 0.42264126, -0.27696662, -0.97017005]],

        [[-1.67807302, -2.3797029 , -2.70265565],
         [ 2.8892855 ,  2.23235788,  1.54374136]],

        ...,

        [[ 1.27861861,  0.58604033,  0.16766436],
         [-0.64750605, -1.18576715, -1.95554604]],

        [[-2.53846207,  2.97397588,  2.24433734],
         [ 1.53576113,  0.80671165,  0.02064281]],

        [[-0.78539611, -1.33518418, -2.17852745],
         [-2.95841921,  2.49822765,  2.07803605]]],


       [[[ 1.85540002,  0.82073941, -0.1487886 ],
         [-1.05547879, -1.81460658, -2.66824409]],

        [[ 2.88325409,  2.1673971 ,  1.46434626],
         [ 0.6836344 , -0.00675187, -0.67935286]],

        [[-1.31873308, -1.98963418, -2.31300997],
         [-3.02353393,  2.67385436,  2.05409032]],

        ...,

        [[-2.94138217,  2.63937561,  2.34003713],
  

In [30]:
a4 = target_sig**4 / (np.abs(target_sig)**3)

In [31]:
np.max(np.angle(a4))

3.1415828342517904

In [33]:
np.angle(a4)

array([[[[-0.94924091,  1.85715378, -1.69446346],
         [ 1.25775725, -2.06563373,  0.99858541]],

        [[-2.00143439,  1.34542114, -1.67617317],
         [ 1.69056504, -1.1078665 ,  2.4025051 ]],

        [[-0.42910677,  3.04755901,  1.75574801],
         [-1.00922863,  2.64624622, -0.10821987]],

        ...,

        [[-1.16871088,  2.3441613 ,  0.67065746],
         [-2.5900242 ,  1.54011671, -1.53899885]],

        [[ 2.41252234, -0.67046709,  2.69416404],
         [-0.14014079, -3.05633869,  0.08257124]],

        [[-3.14158445,  0.94244858, -2.43092448],
         [ 0.73269376, -2.57346003,  2.02895891]]],


       [[[ 1.13841478, -3.00022765, -0.59515438],
         [ 2.06127014, -0.97524103,  1.89339426]],

        [[-1.03335427,  2.38640309, -0.42580026],
         [ 2.73453761, -0.02700748, -2.71741143]],

        [[ 1.00825297, -1.67535142, -2.96885459],
         [ 0.4722349 , -1.87095317,  1.93317596]],

        ...,

        [[ 0.80084194, -2.00886819,  3.07696323],
  